In [3]:
from skopt.sampler import Lhs
import pandas as pd

In [ ]:
def lhs_sampler(n_samples, num_rounds, boundaries, column_names, df):
    

    sampler = Lhs(criterion="maximin", iterations=10000)
    label = 'Maximin LHS'

    dataframes = []

    for i in range(1, num_rounds + 1):
        dataframes.append('df' + f'{i}')
        sampled = sampler.generate(boundaries, n_samples)
        dataframes[i - 1] = pd.DataFrame(sampled, columns = column_names)
        dataframes[i - 1]['Type'] = label + f' r{i}'
        
    df_true = df.copy()
    df_true['Type'] = 'Origin'


    all_points = pd.concat([df_true] + dataframes, ignore_index = True)
    
    return all_points


In [7]:
test_param = [-0.98454527, 0.84028646, -1.01608287, 0.03414988, 9.02499104, 0.45168016, 40.0, 500.0, 1.0]
keys = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 'M_TURN', 't_STAR', 'L_X','NU_X_THRESH', 'X_RAY_SPEC_INDEX']

input_dict = {k:v for k, v in zip(keys, test_param)}

column_names = [r'$f_{\star,10}$', r'$\alpha_\star$', r'$f_{ESC,10}$', r'$\alpha_{ESC}$', r'$M_{TURN}$', r'$t_\star$', r'$L_X$', r'$\nu_{X,THRESH}$', r'$X-ray_{spec,index}$']

df = pd.DataFrame([input_dict])
df.columns = column_names


boundaries = []
for value in df.iloc[0]:
    boundaries.append((value - abs(value) * 0.1, value + abs(value) * 0.1))

In [ ]:
n_samples = 1000
num_rounds = 2

training_data1 = lhs_sampler(n_samples = n_samples, num_rounds = num_rounds, boundaries = boundaries, column_names = column_names, df = df)